In [ ]:
import pandas as pd
import numpy as np


In [ ]:
train_data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
sale_item = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
shop_name = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test_data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
train_data.head()

In [ ]:
def split_month(columns):
    word = columns.split('.')[1]
    return word
train_data['Month'] = train_data['date'].apply(split_month)

#Add the Year columns
def split_year(columns):
    word = columns.split('.')[2]
    return word
train_data['Year'] = train_data['date'].apply(split_year)

#Add the Sales columns
train_data['Sales'] = train_data['item_price']*train_data['item_cnt_day']

#Add the item_categories columns
item_categories = []
for i in train_data['item_id']:
    item_categories.append(sale_item['item_category_id'].iloc[i])

train_data['item_categories'] = item_categories

# Add the item_id_categories columns
train_data['item_id_categories'] = train_data['item_id'].apply(str) + ',' +train_data['item_categories'].apply(str)

In [ ]:
train_Data = train_data[train_data['Month'] == '11']
training_data = train_data.drop(columns = ['date', 'date_block_num', 'item_price', 'Month', 'Year','Sales', 'item_id_categories', 'item_cnt_day'])
training_target = train_data['item_cnt_day']

In [ ]:
training_data = np.array(training_data)
training_target = np.array(training_target)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import plot_model

In [ ]:
model = Sequential()
model.add(Dense(4, activation = 'sigmoid', input_dim = training_data.shape[1]))
model.add(Dense(3, activation = 'relu'))
model.add(Dense(2, activation = 'relu'))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer= 'adam', 
              loss = 'mse', 
              metrics = ['mse', 'mae'])
history = model.fit(training_data, training_target, epochs = 15, batch_size = 512, validation_split = 0.2)

In [ ]:
test_Data = test_data.copy()
print(test_Data.head())
#Add the item_categories columns
item_categor = []
for i in test_Data['item_id']:
    item_categor.append(sale_item['item_category_id'].iloc[i])

test_Data['item_categories'] = item_categor

In [ ]:
test_Data = test_Data.drop(columns = 'ID')
test_Data = np.array(test_Data)
test_data['item_cnt_month'] = model.predict(test_Data)

In [ ]:
submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
submission['item_cnt_month'] = model.predict(test_Data)
submission.to_csv('submission.csv', index=False)